# AI API anomaly detector using graph neural networks models

### Import modules

In [ ]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import ToDense
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.nn import DenseGCNConv
import torch_geometric.utils
from torch_geometric.data import Batch, Data
from torch.nn.functional import pad
from sklearn.model_selection import train_test_split
import networkx as nx
import matplotlib.pyplot as plt
import random
import json
import pandas as pd

### Define the GCN model

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 256)
        self.conv2 = GCNConv(256, 256)
        self.conv3 = GCNConv(256, num_classes)
        self.dropout = torch.nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = x.unsqueeze(0)
        x = self.dropout(F.relu(self.conv1(x, edge_index)))
        x = self.dropout(F.relu(self.conv2(x, edge_index)))
        x = self.conv3(x, edge_index)
        x = x.squeeze(0)
        return F.log_softmax(x, dim=1)

### Load and process graph data

In [ ]:
data_path = 'data'
csv_path = f'{data_path}\supervised_dataset.csv'
json_path = f'{data_path}\supervised_call_graphs.json'

data_frame = pd.read_csv(csv_path)
print(f"Loaded {len(data_frame)} records from CSV file.")
with open(json_path, 'r') as json_data:
  graph_data = json.load(json_data)
print(f"Loaded {len(graph_data)} records from JSON file.")

print("Dataframe columns:", data_frame.columns)
print("Graph data keys:", graph_data[0].keys())
#print("Graph data sample:", graph_data[1])

### Create mappings from IDs to labels

In [ ]:
id_to_label = {item['_id']: (0 if item['classification'] == 'normal' else 1) for item in data_frame.to_dict('records')}

class_counts = pd.Series([id_to_label[_id] for _id in id_to_label]).value_counts() #check the distribution of the classes
print(class_counts)

### Define useful functions

In [ ]:
def create_graph_objects(graph_data, id_to_label):
    graphs = []
    for item in graph_data:
        _id = item['_id']
        
        # Process to ensure edge indices are numeric and consecutive
        node_ids = {}
        edge_indices = []
        for edge in item['call_graph']:
            from_id = edge['fromId']
            to_id = edge['toId']
            
            if from_id not in node_ids:
                node_ids[from_id] = len(node_ids)
            if to_id not in node_ids:
                node_ids[to_id] = len(node_ids)
            
            edge_indices.append([node_ids[from_id], node_ids[to_id]])
        
        # Convert edge_indices list to a tensor
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous() if edge_indices else torch.empty((2, 0), dtype=torch.long)

        # Number of nodes and feature matrix
        num_nodes = len(node_ids)
        num_features = 10  # Choose an appropriate number of features
        x = torch.zeros(num_nodes, num_features)  # Initialize feature matrix with zeros
        for node_id, i in node_ids.items():
            x[i, i % num_features] = 1  # One-hot encoding of node index
        
        # Get the label and repeat it for each node
        label = torch.tensor([id_to_label[_id]] * num_nodes, dtype=torch.long)

        graphs.append(Data(x=x, edge_index=edge_index, y=label, num_nodes=num_nodes))

    return graphs

def collate(data_list):
    return Batch.from_data_list(data_list)

def oversample_minority(graphs):
    # Count the number of instances for each class
    class_counts = pd.Series([graph.y[0].item() for graph in graphs if graph.y.nelement() > 0]).value_counts()
    
    # Identify the minority class
    minority_class = class_counts.idxmin()
    
    # Calculate the difference between the majority and minority class
    diff = class_counts.max() - class_counts.min()
    
    # Get all graphs of the minority class
    minority_graphs = [graph for graph in graphs if graph.y.nelement() > 0 and graph.y[0].item() == minority_class]
    
    # Duplicate the minority graphs until the classes are balanced
    duplicate_graphs = random.choices(minority_graphs, k=diff)
    
    # Add the duplicated graphs to the original list
    graphs += duplicate_graphs

    return graphs

def print_label_distribution(graphs):
    # Count the number of instances for each class
    class_counts = pd.Series([graph.y[0].item() for graph in graphs if graph.y.nelement() > 0]).value_counts()
    
    print(class_counts)

def data_to_networkx(data):
    G = nx.Graph()
    edge_index = data.edge_index.t().numpy()
    num_nodes = data.num_nodes
    for i in range(num_nodes):
        G.add_node(i, features=data.x[i].numpy())
    G.add_edges_from(edge_index)
    return G

def visualize_graphs(graphs):
    num_graphs = len(graphs)
    cols = int(num_graphs**0.5) + 1
    rows = (num_graphs + cols - 1) // cols

    _, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes = axes.flatten() if num_graphs > 1 else [axes]

    for ax, graph_data_object in zip(axes, graphs):
        nx_graph = data_to_networkx(graph_data_object)
        node_labels = {node: f'Node {node}\nLabel {graph_data_object.y[node].item()}' for node in nx_graph.nodes}
         
        pos = nx.spring_layout(nx_graph, seed=42)
        nx.draw(nx_graph, pos, ax=ax, with_labels=True, node_color='skyblue', node_size=700, font_weight='bold', font_color='darkred', labels=node_labels)

    plt.tight_layout()
    plt.show()


### Create graphs

In [ ]:
graphs = create_graph_objects(graph_data, id_to_label)
graphs = oversample_minority(graphs)
torch.set_printoptions(threshold=10_000)

print(f"Created {len(graphs)} graph objects.")
print("Graph object sample:", graphs[0])

### Separate the graphs into two classes

In [ ]:
graphs_class_0 = [graph for graph in graphs if graph.y.nelement() > 0 and graph.y[0].item() == 0]
graphs_class_1 = [graph for graph in graphs if graph.y.nelement() > 0 and graph.y[0].item() == 1]

### Visualise graphs

In [ ]:
visualize_graphs(graphs_class_0[:6])
visualize_graphs(graphs_class_1[:6])

### Perform a train-test split on each class separately

In [ ]:
train_graphs_class_0, val_graphs_class_0 = train_test_split(graphs_class_0, test_size=0.20, random_state=42)
train_graphs_class_1, val_graphs_class_1 = train_test_split(graphs_class_1, test_size=0.20, random_state=42)

### Combine the results

In [ ]:
train_graphs = train_graphs_class_0 + train_graphs_class_1
val_graphs = val_graphs_class_0 + val_graphs_class_1

### Shuffle the data

In [ ]:
random.shuffle(train_graphs)
random.shuffle(val_graphs)

### Create DataLoaders

In [ ]:
train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True, collate_fn=collate)
val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False, collate_fn=collate)
print("Label distribution in all graphs:")
print_label_distribution(graphs)

print("Label distribution in training graphs:")
print_label_distribution(train_graphs)

print("Label distribution in validation graphs:")
print_label_distribution(val_graphs)

### Define the model, optimizer, and loss function

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model = GCN(num_features=graphs[0].num_node_features, num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  # Decrease learning rate
criterion = F.nll_loss

def train():
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()

def validate():
    model.eval()
    correct = 0
    for data in val_loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
        pred = out.argmax(dim=1)
        correct += pred.eq(data.y).sum().item()
    return correct / len(val_dataset)

def print_predictions():
    model.eval()
    for data in val_loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
        pred = out.argmax(dim=1)
        print(f'Predictions: {pred}')
        print(f'Actual labels: {data.y}')


val_dataset = val_graphs

for epoch in range(10):
    train()
    val_acc = validate()
    print(f'Epoch: {epoch}, Validation Accuracy: {val_acc}')

print_predictions()